In [1]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mjawad1","key":"e80a08bf3100d35cd537ea408d3242b0"}'}

In [2]:
!mkdir -p /root/.config/kaggle
!mv kaggle.json /root/.config/kaggle/
!chmod 600 /root/.config/kaggle/kaggle.json


In [3]:
!kaggle datasets list


ref                                                       title                                               size  lastUpdated                 downloadCount  voteCount  usabilityRating  
--------------------------------------------------------  --------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
wardabilal/spotify-global-music-dataset-20092025          Spotify Global Music Dataset (2009–2025)         1289021  2025-11-11 09:43:05.933000          14547        331  1.0              
rohiteng/amazon-sales-dataset                             Amazon Sales Dataset                             4037578  2025-11-23 14:29:37.973000           5477         75  1.0              
rohiteng/spotify-music-analytics-dataset-20152025         Spotify Music Analytics Dataset (2015–2025)      5080400  2025-12-04 05:17:45.537000           1652         41  1.0              
kundanbedmutha/exam-score-prediction-dataset              Ex

In [4]:
!kaggle datasets download -d kushagratandon12/diabetic-retinopathy-balanced


Dataset URL: https://www.kaggle.com/datasets/kushagratandon12/diabetic-retinopathy-balanced
License(s): CC0-1.0
 98% 1.85G/1.90G [00:16<00:00, 102MB/s] 
100% 1.90G/1.90G [00:16<00:00, 121MB/s]


In [5]:
!unzip -q diabetic-retinopathy-balanced.zip -d data


In [6]:
!ls data


content


In [7]:
!ls data/content


Diabetic_Balanced_Data


In [8]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten,
    Dense, Dropout, BatchNormalization
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [9]:
IMG_SIZE   = (128,128)
BATCH_SIZE = 64
EPOCHS = 20

In [24]:
datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range= 10,
    zoom_range = 0.1,
    horizontal_flip = True,
    # brightness_range = [0.8,1.2],
    validation_split = 0.2

)

In [11]:
train_data = datagen.flow_from_directory(
    "data/content/Diabetic_Balanced_Data/train",
    target_size = IMG_SIZE,
    batch_size = BATCH_SIZE,
    class_mode = "categorical",
    subset = "training"
)

Found 27834 images belonging to 5 classes.


In [12]:
val_data = datagen.flow_from_directory(
    "data/content/Diabetic_Balanced_Data/train",
    target_size = IMG_SIZE,
    batch_size = BATCH_SIZE,
    class_mode = "categorical",
    subset = "validation",
    shuffle = False

)

Found 6958 images belonging to 5 classes.


In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, 3, activation='relu', input_shape=(128,128,3)),
    MaxPooling2D(),

    Conv2D(64, 3, activation='relu'),
    MaxPooling2D(),

    Conv2D(128, 3, activation='relu'),
    MaxPooling2D(),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(5, activation='softmax')
])

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,305,285 (12.61 MB)

 Trainable params: 3,305,285 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss = "categorical_crossentropy",
    metrics = ['accuracy']
)

In [15]:
early_stop = EarlyStopping(
  monitor = "val_loss",
  patience = 3,
  restore_best_weights = True

)

In [16]:
checkpoint = ModelCheckpoint(
    "best_model.keras",
    monitor = "val_loss",
    save_best_only=True

)

In [17]:
history = model.fit(
    train_data,
    validation_data = val_data,
    epochs = EPOCHS,
    callbacks = [early_stop, checkpoint]
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
435/435 ━━━━━━━━━━━━━━━━━━━━ 1211s 3s/step - accuracy: 0.2769 - loss: 1.5461 - val_accuracy: 0.4005 - val_loss: 1.3863
Epoch 2/20
435/435 ━━━━━━━━━━━━━━━━━━━━ 1242s 3s/step - accuracy: 0.3798 - loss: 1.3916 - val_accuracy: 0.4330 - val_loss: 1.3328
Epoch 3/20
435/435 ━━━━━━━━━━━━━━━━━━━━ 1279s 3s/step - accuracy: 0.4038 - loss: 1.3358 - val_accuracy: 0.4408 - val_loss: 1.2934
Epoch 4/20
435/435 ━━━━━━━━━━━━━━━━━━━━ 1275s 3s/step - accuracy: 0.4076 - loss: 1.3224 - val_accuracy: 0.4404 - val_loss: 1.2826
Epoch 5/20
435/435 ━━━━━━━━━━━━━━━━━━━━ 1261s 3s/step - accuracy: 0.4192 - loss: 1.2899 - val_accuracy: 0.4014 - val_loss: 1.3613
Epoch 6/20
435/435 ━━━━━━━━━━━━━━━━━━━━ 1229s 3s/step - accuracy: 0.4402 - loss: 1.2789 - val_accuracy: 0.4503 - val_loss: 1.2893
Epoch 7/20
416/435 ━━━━━━━━━━━━━━━━━━━━ 47s 3s/step - accuracy: 0.4359 - loss: 1.2594

KeyboardInterrupt: 

In [20]:
model.save("custom_dr_model.h5")
from google.colab import files
files.download("custom_dr_model.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
from sklearn.metrics import classification_report

val_data.reset()
y_pred = model.predict(val_data)
y_pred_classes = np.argmax(y_pred, axis = 1)
y_true = val_data.classes

print(classification_report(y_true, y_pred_classes))

109/109 ━━━━━━━━━━━━━━━━━━━━ 108s 987ms/step
              precision    recall  f1-score   support

           0       0.44      0.73      0.55      1400
           1       0.35      0.38      0.36      1358
           2       0.35      0.17      0.23      1400
           3       0.53      0.43      0.48      1400
           4       0.61      0.56      0.59      1400

    accuracy                           0.46      6958
   macro avg       0.46      0.46      0.44      6958
weighted avg       0.46      0.46      0.44      6958



In [22]:
def grad_cam(model, img_array, layer_name):
    grad_model = tf.keras.models.Model(
        model.inputs,
        [model.get_layer(layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, preds = grad_model(img_array)
        loss = preds[:, np.argmax(preds[0])]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0,1,2))

    heatmap = tf.reduce_sum(conv_outputs * pooled_grads, axis=-1)
    heatmap = np.maximum(heatmap, 0) / tf.reduce_max(heatmap)

    return heatmap[0].numpy()
